In [169]:
import requests
from bs4 import BeautifulSoup
from PIL import Image, ImageDraw, ImageFont
import copy
import textwrap
from pathlib import WindowsPath
from datetime import date
import os


In [170]:
today = date.today()
# today = '2024-01-26'

In [171]:
# content_dic['DRUGIE CZYTANIE'][4] = 'Łaska wam i pokój od Boga ojca naszego i Pana Jezusa Chrystusa! Bogu mojemu dziękuję wciąż za was, za łaskę daną wam w Chrystusie Jezusie. W Nim to bowiem zostaliście wzbogaceni we wszystko: we wszelkie słowo i wszelkie poznanie, bo świadectwo Chrystusowe utrwaliło się w was. Nie doznajecie tedy braku żadnej łaski, oczekując objawienia się Pana naszego, Jezusa Chrystusa. On też będzie umacniał was aż do końca, abyście byli bez zarzutu w dzień Pana naszego, Jezusa Chrystusa.'
# content_dic['EWANGELIA'][3] = 'Jezus powiedział do swoich uczniów:'
# content_dic['EWANGELIA'].insert(4,'«Uważajcie, czuwajcie, bo nie wiecie, kiedy czas ten nadejdzie. Bo rzecz ma się podobnie jak z człowiekiem, który udał się w podróż. Zostawił swój dom, powierzył swym sługom staranie o wszystko, każdemu wyznaczył zajęcie, a odźwiernemu przykazał, żeby czuwał.')
# content_dic['EWANGELIA']

In [172]:
URL = f"https://liturgia.wiara.pl/kalendarz/67b53.Czytania-mszalne/{str(today)}"


page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

content_list = soup.find_all("div", "txt__rich-area")[1].get_text().split('\n')

if content_list[-1] == '':
    del content_list[-1]

In [173]:
content_list

['',
 'Liturgia Słowa wg lekcjonarza z 2015 rokuLiturgia Słowa do odsłuchania (wersja wideo)Liturgia Słowa według starego lekcjonarza',
 'Liturgia Słowa wg lekcjonarza z 2015 roku',
 'PIERWSZE CZYTANIE',
 '1 Krl 3, 4-13',
 'Bóg wysłuchuje prośby Salomona',
 'Czytanie z Pierwszej Księgi Królewskiej',
 'Król Salomon udał się do Gibeonu, aby tam złożyć ofiarę, bo była tam wielka wyżyna. Salomon złożył na owym ołtarzu tysiąc ofiar całopalnych. W Gibeonie Pan ukazał się Salomonowi w nocy, we śnie. Wtedy Bóg rzekł: «Proś o to, co mam ci dać».',
 'A Salomon odrzekł: «Ty okazywałeś Twemu słudze, Dawidowi, memu ojcu, wielką łaskę, bo postępował wobec Ciebie szczerze, sprawiedliwie i w prostocie serca. Ponadto zachowałeś dla niego tę wielką łaskę, że dałeś mu syna zasiadającego na jego tronie po dziś dzień. Teraz więc, o Panie, Boże mój, Ty ustanowiłeś królem Twego sługę w miejsce Dawida, mego ojca, a ja jestem bardzo młody i nie umiem rządzić. Ponadto Twój sługa jest pośród Twego ludu, który wy

In [174]:
URL

'https://liturgia.wiara.pl/kalendarz/67b53.Czytania-mszalne/2024-02-03'

In [175]:
# content_list[content_list.index('EWANGELIA')]
# content_dic['PIERWSZE CZYTANIE'] = content_dic['PIERWSZE CZYTANIE'][:-5]

# content_dic['PSALM RESPONSORYJNY'].insert(11, 'Refren: Po całej ziemi ich głos się rozchodzi.')
# content_dic['PSALM RESPONSORYJNY']

In [176]:
if 'EWANGELIA DŁUŻSZA' in content_list:
    cut_points = ['PIERWSZE CZYTANIE', 'PSALM RESPONSORYJNY', 'DRUGIE CZYTANIE', 'EWANGELIA DŁUŻSZA', 'EWANGELIA KRÓTSZA']
elif 'DRUGIE CZYTANIE' in content_list:
    cut_points = ['PIERWSZE CZYTANIE', 'PSALM RESPONSORYJNY', 'DRUGIE CZYTANIE', 'EWANGELIA']
else:
    cut_points = ['PIERWSZE CZYTANIE', 'PSALM RESPONSORYJNY', 'EWANGELIA']

content_dic = {}
staging_list = [] 


for text in content_list:
    if 'Liturgia Słowa' in text or '' == text:
        continue

    # content_list[content_list.index(text)] = text.strip()
    # text = text.strip()
    
    if text in cut_points:

        for text in content_list[content_list.index(text)+1:]:
            if '' == text:
                continue
            if text == 'albo':
                content_dic[cut_points[0]] = staging_list
                staging_list = [] 
                del cut_points[0]
                break
            if text not in cut_points:
                staging_list.append(text)
                
            else:
                content_dic[cut_points[0]] = staging_list
                staging_list = [] 
                del cut_points[0]
                break
        
        content_dic[cut_points[0]] = staging_list

content_dic 

        

{'PIERWSZE CZYTANIE': ['1 Krl 3, 4-13',
  'Bóg wysłuchuje prośby Salomona',
  'Czytanie z Pierwszej Księgi Królewskiej',
  'Król Salomon udał się do Gibeonu, aby tam złożyć ofiarę, bo była tam wielka wyżyna. Salomon złożył na owym ołtarzu tysiąc ofiar całopalnych. W Gibeonie Pan ukazał się Salomonowi w nocy, we śnie. Wtedy Bóg rzekł: «Proś o to, co mam ci dać».',
  'A Salomon odrzekł: «Ty okazywałeś Twemu słudze, Dawidowi, memu ojcu, wielką łaskę, bo postępował wobec Ciebie szczerze, sprawiedliwie i w prostocie serca. Ponadto zachowałeś dla niego tę wielką łaskę, że dałeś mu syna zasiadającego na jego tronie po dziś dzień. Teraz więc, o Panie, Boże mój, Ty ustanowiłeś królem Twego sługę w miejsce Dawida, mego ojca, a ja jestem bardzo młody i nie umiem rządzić. Ponadto Twój sługa jest pośród Twego ludu, który wybrałeś, ludu mnogiego, którego nie da się zliczyć ani też spisać z powodu jego mnóstwa. Racz więc dać Twemu słudze serce rozumne do sądzenia Twego ludu i rozróżniania dobra od zł

In [177]:
content_dic.keys()


dict_keys(['PIERWSZE CZYTANIE', 'PSALM RESPONSORYJNY', 'EWANGELIA'])

In [178]:
if 'EWANGELIA KRÓTSZA' in content_list:
    content_dic['EWANGELIA KRÓTSZA'].insert(1, content_dic['EWANGELIA DŁUŻSZA'][1])
    content_dic['EWANGELIA KRÓTSZA']

In [179]:
from functools import reduce
count = reduce(lambda x, y: x + 1 if 'Refren' in y else x, content_dic['PSALM RESPONSORYJNY'], 0)
if len(content_dic['PSALM RESPONSORYJNY']) > 6 and count == 1:
    content_dic['PSALM RESPONSORYJNY'].insert(6, 'Refren')
    content_dic['PSALM RESPONSORYJNY'].insert(11, 'Refren')
    content_dic['PSALM RESPONSORYJNY'].insert(16, 'Refren')
content_dic['PSALM RESPONSORYJNY']

['Ps 119 (118), 9-10. 11-12. 13-14 (R.: por. 12b)',
 'Refren: Naucz mnie, Panie, mądrych ustaw swoich.',
 'Jak zachowa młodzieniec swoją drogę w czystości? *',
 'Przestrzegając słów Twoich, Panie.',
 'Z całego serca swego szukam Ciebie, *',
 'nie daj mi odejść od Twoich przykazań.',
 'Refren.',
 'W sercu swoim zachowuję Twe słowa, *',
 'aby nie zgrzeszyć przeciw Tobie.',
 'Błogosławiony jesteś, Panie, *',
 'naucz mnie swoich ustaw.',
 'Refren.',
 'Opowiadam moimi wargami *',
 'wszystkie wyroki ust Twoich.',
 'Więcej się cieszę z drogi wskazanej',
 'przez Twe napomnienia *',
 'niż z wszelkiego bogactwa.',
 'Refren.',
 'ŚPIEW PRZED EWANGELIĄ',
 'Por. J 10, 27',
 'Aklamacja: Alleluja, alleluja, alleluja.',
 'Moje owce słuchają mego głosu,',
 'Ja znam je, a one idą za Mną.',
 'Aklamacja: Alleluja, alleluja, alleluja.']

In [180]:
path = r"C:\Users\Acne\Pictures\Cyfrowa Biblia\automatyzacja\paper.gif"

image_size = 1080
with Image.open(path) as im:
    out = Image.new("RGB", (image_size, image_size), "white")
    x=0
    y=0
    size_x_left, size_y = im.size

    while x < image_size:
        while y < image_size:
            out.paste(im, (x, y))
            y += size_y
        y = 0
        x += size_x_left

# out.show()


In [181]:
tahoma = r"C:\Windows\Fonts\tahoma.ttf"
tahoma_bold = r"C:\Windows\Fonts\tahomabd.ttf"

In [182]:
current_path = WindowsPath(r'C:\Users\Acne\Pictures\Cyfrowa Biblia\\' + str(today))

os.makedirs(current_path, exist_ok=True)

In [183]:
def draw_text(out, name, current_path, size_x_left, size_y, font_size):

    font_size_small = int(0.75 * font_size)

    fnt = ImageFont.truetype(tahoma, font_size) 
    fnt_b = ImageFont.truetype(tahoma_bold, font_size)
    fnt_s = ImageFont.truetype(tahoma, font_size_small)

    size_x_left = int(25/12 * font_size)
    size_x_right = image_size - 2 * size_x_left  
    size_y = int(1.2 * font_size)
    width = int(80 - 0.35 * size_x_left)

    out_func = copy.deepcopy(out)
    draw = ImageDraw.Draw(out_func) 

    draw.text((size_x_left, size_x_left), name, font=fnt_b, fill="red", anchor='lm')
    draw.text((size_x_right + size_x_left, size_x_left), content_dic[name][0], font=fnt_b, fill="red", anchor='rm') #TODO
    
    var_y = size_x_left + size_y
    draw.text((size_x_left, var_y), content_dic[name][1], font=fnt_s, fill="black", anchor='lm')
    
    lines = textwrap.wrap(content_dic[name][2], width=width)
    for line in lines:
        var_y += size_y
        draw.text((size_x_left, var_y), line, font=fnt_b, fill="black", anchor='lm')
    
    var_y += size_x_left
    
    for count, element in enumerate(content_dic[name][3:-1]):
        lines = textwrap.wrap(content_dic[name][3+count], width=width)
        print('\n'.join(lines))
        space_length_regular = 15

        for line_count, line in enumerate(lines):
            x_word = size_x_left
            x_word_intended = 2 * size_x_left
            words = line.split(" ")
            words_length = sum(draw.textlength(w, font=fnt) for w in words)
            if len(words) == 1:
                words.append(' ')

            if line_count == len(lines)-1:
                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
                var_y += size_y  
                break
            
            space_length_intended = (size_x_right - x_word - words_length) / (len(words) - 1)
            space_length_regular = (size_x_right - words_length) / (len(words) - 1)
           
            if line_count == 0 and line != 'Bracia:' and 'Jezus powiedział' not in line:
                for word in words:
                    draw.text((x_word_intended, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word_intended += draw.textlength(word, font=fnt) + space_length_intended

            else:
                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
            var_y += size_y            
            
    draw.text((2*size_x_left, var_y+size_y), content_dic[name][-1], font=fnt_b, fill="black", anchor='lm')

    # out_func.show()

    returned = {'drawn_y': var_y+size_y, 'picture': out_func}

    return returned
    # out_func.save(current_path / f'{name}.png')

    out_func = copy.deepcopy(out)

returned = draw_text(out, text, current_path, size_x_left, size_y, 44)
returned['picture'].show()

KeyError: 'Oto słowo Pańskie.'

In [ ]:
if "DRUGIE CZYTANIE" in content_dic:
    if 'ŚPIEW PRZED EWANGELIĄ' in content_dic["DRUGIE CZYTANIE"]:
        content_dic["DRUGIE CZYTANIE"] = content_dic["DRUGIE CZYTANIE"][:-6]

In [ ]:
returned = draw_text(out, text, current_path, size_x_left, size_y, 4)
returned['picture'].show()

In [185]:
def draw_text_pagination_first(out, reading_list, current_path, size_x_left, size_y, font_size):

    font_size_small = int(0.75 * font_size)

    fnt = ImageFont.truetype(tahoma, font_size) 
    fnt_b = ImageFont.truetype(tahoma_bold, font_size)
    fnt_s = ImageFont.truetype(tahoma, font_size_small)

    size_x_left = int(25/12 * font_size)
    size_x_right = image_size - 2 * size_x_left
    size_y = int(1.2 * font_size)
    width = int(80 - 0.3 * size_x_left)

    out_func = copy.deepcopy(out)
    draw = ImageDraw.Draw(out_func) 

#TODO
    draw.text((size_x_left, size_x_left), reading_list[0], font=fnt_b, fill="red", anchor='lm')
    draw.text((size_x_right + size_x_left, size_x_left), reading_list[1], font=fnt_b, fill="red", anchor='rm') #TODO
    
    var_y = size_x_left + size_y
    draw.text((size_x_left, var_y), reading_list[2], font=fnt_s, fill="black", anchor='lm')
    
    lines = textwrap.wrap(reading_list[3], width=width)
    for line in lines:
        var_y += size_y
        draw.text((size_x_left, var_y), line, font=fnt_b, fill="black", anchor='lm')
    
    var_y += size_x_left
    
    for count, element in enumerate(reading_list[4:]):
        lines = textwrap.wrap(reading_list[4+count], width=width)
        space_length_regular = 15

        for line_count, line in enumerate(lines):
            x_word = size_x_left
            x_word_intended = 2 * size_x_left
            words = line.split(" ")
            words_length = sum(draw.textlength(w, font=fnt) for w in words)
            if len(words)==1:
                words.append(' ')

                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
                var_y += size_y  
                break

            if line_count == len(lines)-1:
                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
                var_y += size_y  
                break
            
            space_length_intended = (size_x_right - x_word - words_length) / (len(words) - 1)
            space_length_regular = (size_x_right - words_length) / (len(words) - 1)
           
            if line_count == 0 and line != 'Bracia:' and 'Jezus powiedział' not in line:
                for word in words:
                    draw.text((x_word_intended, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word_intended += draw.textlength(word, font=fnt) + space_length_intended

            else:
                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
            var_y += size_y            
            
    # out_func.show()

    returned = {'drawn_y': var_y+size_y, 'picture': out_func}

    return returned
    # out_func.save(current_path / f'{name}.png')

    out_func = copy.deepcopy(out)

In [191]:
def draw_text_pagination_second(out, reading_list, current_path, size_x_left, size_y, font_size):

    font_size_small = int(0.75 * font_size)
 
    fnt = ImageFont.truetype(tahoma, font_size) 
    fnt_b = ImageFont.truetype(tahoma_bold, font_size)
    fnt_s = ImageFont.truetype(tahoma, font_size_small)

    size_x_left = int(25/12 * font_size)
    size_x_right = image_size - 2 * size_x_left
    size_y = int(1.2 * font_size)
    width = int(80 - 0.3 * size_x_left)

    out_func = copy.deepcopy(out)
    draw = ImageDraw.Draw(out_func) 

    var_y = size_x_left
    print(reading_list)
    lines = textwrap.wrap(reading_list[0], width=width)
    
    for count, element in enumerate(reading_list[0:-1]):
        lines = textwrap.wrap(reading_list[0+count], width=width)
        space_length_regular = 15

        for line_count, line in enumerate(lines):
            x_word = size_x_left
            x_word_intended = 2 * size_x_left
            words = line.split(" ")
            words_length = sum(draw.textlength(w, font=fnt) for w in words)
            if len(words)==1:
                words.append(' ')

                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
                var_y += size_y  
                break

            if line_count == len(lines)-1:
                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
                var_y += size_y  
                break
            
            space_length_intended = (size_x_right - x_word - words_length) / (len(words) - 1)
            space_length_regular = (size_x_right - words_length) / (len(words) - 1)
           
            if line_count == 0 and line != 'Bracia:' and 'Jezus powiedział' not in line:
                for word in words:
                    draw.text((x_word_intended, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word_intended += draw.textlength(word, font=fnt) + space_length_intended

            else:
                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
            var_y += size_y            
            
    draw.text((2*size_x_left, var_y+size_y), reading_list[-1], font=fnt_b, fill="black", anchor='lm')

    # out_func.show()

    returned = {'drawn_y': var_y+size_y, 'picture': out_func}

    return returned
    # out_func.save(current_path / f'{name}.png')

    out_func = copy.deepcopy(out)

In [ ]:
content_dic.keys()


dict_keys(['PIERWSZE CZYTANIE', 'PSALM RESPONSORYJNY', 'EWANGELIA'])

In [192]:

min_font_before_pagination = 29
verse_break = 7
pagination_font_size = 34


for text in content_dic.keys():
# ['PIERWSZE CZYTANIE']:
    if text != 'PSALM RESPONSORYJNY':
        font_size = 42
        returned = draw_text(out, text, current_path, size_x_left, size_y, font_size)

        returned['picture'].show()
        returned['picture'].save(current_path / f'{text}.png')

        while returned['drawn_y'] > image_size - 20:
            if font_size < min_font_before_pagination:
                pagination_dic = {}
                pagination_dic[f'{text} cz.1'] = content_dic[f'{text}'][:verse_break]
                pagination_dic[f'{text} cz.1'].insert(0, f'{text}' )
                pagination_dic[f'{text} cz.2'] = content_dic[f'{text}'][verse_break:]

                
                returned = draw_text_pagination_first(out, pagination_dic[f'{text} cz.1'], current_path, size_x_left, size_y, pagination_font_size)
                returned['picture'].show()
                returned['picture'].save(current_path / f'{text}1.png')

                returned = draw_text_pagination_second(out,  pagination_dic[f'{text} cz.2'], current_path, size_x_left, size_y, pagination_font_size)
                returned['picture'].show()
                returned['picture'].save(current_path / f'{text}2.png')

                break
            
            else:
                font_size -= 1
                returned = draw_text(out, text, current_path, size_x_left, size_y, font_size)
        
        print('font_size: ', font_size)
        
        

Król Salomon udał się do Gibeonu, aby tam złożyć
ofiarę, bo była tam wielka wyżyna. Salomon złożył
na owym ołtarzu tysiąc ofiar całopalnych. W
Gibeonie Pan ukazał się Salomonowi w nocy, we
śnie. Wtedy Bóg rzekł: «Proś o to, co mam ci
dać».
A Salomon odrzekł: «Ty okazywałeś Twemu słudze,
Dawidowi, memu ojcu, wielką łaskę, bo postępował
wobec Ciebie szczerze, sprawiedliwie i w
prostocie serca. Ponadto zachowałeś dla niego tę
wielką łaskę, że dałeś mu syna zasiadającego na
jego tronie po dziś dzień. Teraz więc, o Panie,
Boże mój, Ty ustanowiłeś królem Twego sługę w
miejsce Dawida, mego ojca, a ja jestem bardzo
młody i nie umiem rządzić. Ponadto Twój sługa
jest pośród Twego ludu, który wybrałeś, ludu
mnogiego, którego nie da się zliczyć ani też
spisać z powodu jego mnóstwa. Racz więc dać Twemu
słudze serce rozumne do sądzenia Twego ludu i
rozróżniania dobra od zła, bo któż zdoła sądzić
ten lud Twój tak liczny?»
Spodobało się Panu, że właśnie o to Salomon
poprosił. Bóg więc mu powiedział: «

IndexError: list index out of range

In [ ]:
# returned = draw_text(out, 'PIERWSZE CZYTANIE', current_path, size_x_left, size_y, 26)
# returned['picture'].show()
# returned['picture'].save(current_path / f'{text}.png')

In [ ]:
font_size_psalm = 32
font_size_small_psalm = int(0.75 * font_size_psalm)

tahoma = r"C:\Windows\Fonts\tahoma.ttf"
tahoma_bold = r"C:\Windows\Fonts\tahomabd.ttf"

fnt_psalm = ImageFont.truetype(tahoma, font_size_psalm) 
fnt_b_psalm = ImageFont.truetype(tahoma_bold, font_size_psalm)
fnt_s_psalm = ImageFont.truetype(tahoma, font_size_small_psalm)

In [ ]:
# content_dic["PSALM RESPONSORYJNY"].insert(16, 'Refren')
content_dic["PSALM RESPONSORYJNY"]

['Ps 96,1-2.3.7-8a i 10',
 'Refren: Głoście cześć Pana wśród wszystkich narodów.',
 'Śpiewajcie Panu pieśń nową',
 'śpiewaj Panu, ziemio cała.',
 'Śpiewajcie Panu, sławcie Jego imię,',
 'każdego dnia głoście Jego zbawienie.',
 'Refren',
 'Głoście Jego chwałę',
 'wśród wszystkich narodów,',
 'rozgłaszajcie Jego cuda',
 'pośród wszystkich ludów.',
 'Refren',
 'Oddajcie Panu, rodziny narodów,',
 'oddajcie Panu chwałę i uznajcie Jego potęgę.',
 'Oddajcie Panu chwałę należną Jego imieniu.',
 'Głoście wśród ludów, że Pan jest królem.',
 'Refren',
 'utwierdził świat. tak, że się nie zachwieje,',
 'będzie sprawiedliwie sądził ludy.',
 'ŚPIEW PRZED EWANGELIĄ',
 'Łk 4,18',
 'Alleluja, alleluja, alleluja',
 'Pan posłał Mnie, abym ubogim głosił dobrą nowinę,',
 'więźniom głosił wolność.',
 'Alleluja, alleluja, alleluja']

In [ ]:
print(len(content_dic["PSALM RESPONSORYJNY"][0]))
if len(content_dic["PSALM RESPONSORYJNY"][0]) > 35:
    content_dic["PSALM RESPONSORYJNY"][0] = content_dic["PSALM RESPONSORYJNY"][0].split(',')[0]
    
# del content_dic["PSALM RESPONSORYJNY"][-2]
# content_dic["PSALM RESPONSORYJNY"].insert(-1, 'pójdźcie, narody, oddajcie pokłon Panu,')
# content_dic["PSALM RESPONSORYJNY"].insert(-1, 'bo wielka światłość zstąpiła dzisiaj na ziemię.')
content_dic["PSALM RESPONSORYJNY"]

21


['Ps 96,1-2.3.7-8a i 10',
 'Refren: Głoście cześć Pana wśród wszystkich narodów.',
 'Śpiewajcie Panu pieśń nową',
 'śpiewaj Panu, ziemio cała.',
 'Śpiewajcie Panu, sławcie Jego imię,',
 'każdego dnia głoście Jego zbawienie.',
 'Refren',
 'Głoście Jego chwałę',
 'wśród wszystkich narodów,',
 'rozgłaszajcie Jego cuda',
 'pośród wszystkich ludów.',
 'Refren',
 'Oddajcie Panu, rodziny narodów,',
 'oddajcie Panu chwałę i uznajcie Jego potęgę.',
 'Oddajcie Panu chwałę należną Jego imieniu.',
 'Głoście wśród ludów, że Pan jest królem.',
 'Refren',
 'utwierdził świat. tak, że się nie zachwieje,',
 'będzie sprawiedliwie sądził ludy.',
 'ŚPIEW PRZED EWANGELIĄ',
 'Łk 4,18',
 'Alleluja, alleluja, alleluja',
 'Pan posłał Mnie, abym ubogim głosił dobrą nowinę,',
 'więźniom głosił wolność.',
 'Alleluja, alleluja, alleluja']

In [ ]:
out_psalm = copy.deepcopy(out)
draw = ImageDraw.Draw(out_psalm) 

y_distance = font_size_psalm * 0.95
y_further_distance = font_size_psalm * 1.2
x_distance = font_size_psalm * 2

draw.text((x_distance, y_further_distance*2), "PSALM RESPONSORYJNY", font=fnt_b_psalm, fill="red", anchor='lm')
draw.text((1000, y_further_distance*2), content_dic["PSALM RESPONSORYJNY"][0], font=fnt_b_psalm, fill="red", anchor='rm')
draw.text((x_distance, y_further_distance*3), "Refren: ", font=fnt_b_psalm, fill="red", anchor='lm')
draw.text((x_distance*3, y_further_distance*3), content_dic["PSALM RESPONSORYJNY"][1][7:], font=fnt_b_psalm, fill="black", anchor='lm')

# draw.text((100, 300), content_dic["PSALM RESPONSORYJNY"][3], font=fnt_psalm, fill="black", anchor='lm')
y_text = y_further_distance*4
for count, element in enumerate(content_dic["PSALM RESPONSORYJNY"][2:]):
    # lines = textwrap.wrap(second_element[2+count], width=80)
    # for line in lines:
    if "Refren" in element :
        y_text += y_further_distance
        draw.text((x_distance, y_text), "Refren: ", font=fnt_b_psalm, fill="red", anchor='lm')
        draw.text((x_distance*3, y_text), content_dic["PSALM RESPONSORYJNY"][1][7:], font=fnt_b_psalm, fill="black", anchor='lm')
        y_text += y_further_distance
    elif "ŚPIEW PRZED EWANGELIĄ" in element:
        # print("NUMER: ", inner_count)
        acclamation = content_dic["PSALM RESPONSORYJNY"][count+2:]
        if 'ŚPIEW' in acclamation[0]:
            acclamation.insert(1, '')
        print(acclamation)
        draw.text((x_distance, y_text), "AKLAMACJA PRZED EWANGELIĄ", font=fnt_b_psalm, fill="red", anchor='lm')
        draw.text((1000, y_text), acclamation[1], font=fnt_b_psalm, fill="red", anchor='rm')
        draw.text((x_distance, y_text+y_further_distance), "Aklamacja: ", font=fnt_b_psalm, fill="red", anchor='lm')
        draw.text((x_distance*4, y_text+y_further_distance), acclamation[2][10:], font=fnt_b_psalm, fill="black", anchor='lm')
        draw.text((x_distance*2, y_text+y_further_distance*2), acclamation[3], font=fnt_psalm, fill="black", anchor='lm')
        draw.text((x_distance*2, y_text+y_further_distance*2+y_distance), acclamation[4], font=fnt_psalm, fill="black", anchor='lm')
        # draw.text((x_distance*2, y_text+y_further_distance*3+y_distance), acclamation[5], font=fnt_psalm, fill="black", anchor='lm')

        draw.text((x_distance, y_text+y_further_distance*3+y_distance), "Aklamacja: ", font=fnt_b_psalm, fill="red", anchor='lm') 
        draw.text((x_distance*4, y_text+y_further_distance*3+y_distance), acclamation[5][10:], font=fnt_b_psalm, fill="black", anchor='lm')

        break  

# TODO
    elif 'albo' in element:
        continue

    else:
        draw.text((x_distance*2, y_text), element, font=fnt_psalm, fill="black", anchor='lm')
    y_text += y_distance

out_psalm.show()
out_psalm.save(current_path / 'psalm.png')
# out_psalm.save('./2024-01-08/psalm.png')

out_psalm = copy.deepcopy(out)

['ŚPIEW PRZED EWANGELIĄ', '', 'Łk 4,18', 'Alleluja, alleluja, alleluja', 'Pan posłał Mnie, abym ubogim głosił dobrą nowinę,', 'więźniom głosił wolność.', 'Alleluja, alleluja, alleluja']


In [ ]:
# os.makedirs('./2024-01-08/', exist_ok=True)

In [ ]:
current_path

WindowsPath('C:/Users/Acne/Pictures/Cyfrowa Biblia/2024-01-26')

In [ ]:
# content_dic["EWANGELIA"]

In [ ]:
content_dic.keys()

dict_keys(['PIERWSZE CZYTANIE', 'PSALM RESPONSORYJNY', 'EWANGELIA'])

In [ ]:
# content_dic['PIERWSZE CZYTANIE cz.1'] = content_dic['PIERWSZE CZYTANIE'][:6]
# content_dic['PIERWSZE CZYTANIE cz.2'] = content_dic['PIERWSZE CZYTANIE'][6:]
# returned = draw_text_pagination(out, 'PIERWSZE CZYTANIE cz.1', current_path, size_x_left, size_y, 32)
# returned['picture'].show()
# returned['picture'].save(current_path / f'{text}1.png')
# returned = draw_text_pagination_second(out, 'PIERWSZE CZYTANIE cz.2', current_path, size_x_left, size_y, 32)
# returned['picture'].show()
# returned['picture'].save(current_path / f'{text}2.png')

In [ ]:
# text = 'PIERWSZE CZYTANIE'
# content_dic[f'{text} cz.1'] = content_dic[f'{text}'][:6]
# content_dic[f'{text} cz.2'] = content_dic[f'{text}'][6:]
# returned = draw_text_pagination_first(out, f'{text} cz.1', current_path, size_x_left, size_y, 30)
# returned['picture'].show()
# returned['picture'].save(current_path / f'{text}1.png')

# returned = draw_text_pagination_second(out, f'{text} cz.2', current_path, size_x_left, size_y, 32)
# returned['picture'].show()
# returned['picture'].save(current_path / f'{text}2.png')

In [ ]:
# content_dic['EWANGELIA'] = content_dic['EWANGELIA ']

In [ ]:
# text = 'EWANGELIA'
# content_dic['EWANGELIA cz.1'] = content_dic['EWANGELIA'][:7]
# content_dic['EWANGELIA cz.2'] = content_dic['EWANGELIA'][7:]
# returned = draw_text_pagination_first(out, 'EWANGELIA cz.1', current_path, size_x_left, size_y, 32)
# returned['picture'].show()
# returned['picture'].save(current_path / f'{text}1.png')

# returned = draw_text_pagination_second(out, 'EWANGELIA cz.2', current_path, size_x_left, size_y, 32)
# returned['picture'].show()
# returned['picture'].save(current_path / f'{text}2.png')

In [ ]:
content_dic['PIERWSZE CZYTANIE']

['2 Tm 1,1-8',
 'Żywa wiara Tymoteusza',
 'Czytanie z Drugiego listu świętego Pawła Apostoła do Tymoteusza',
 'Paweł, z woli Boga apostoł Chrystusa Jezusa, posłany dla głoszenia życia obiecanego w Chrystusie Jezusie, do Tymoteusza, swego umiłowanego dziecka. Łaska, miłosierdzie, pokój od Boga Ojca i Chrystusa Jezusa, naszego Pana.',
 'Dziękuję Bogu, któremu służę jak moi przodkowie z czystym sumieniem, gdy zachowuję nieprzerwaną pamięć o tobie w moich modlitwach. W nocy i we dnie pragnę cię zobaczyć, pomny na twoje łzy, by napełniła mnie radość na wspomnienie wiary bez obłudy, jaka jest w tobie; ona to zamieszkała pierwej w twej babce Lois i w twej matce Eunice, a pewien jestem, że mieszka i w tobie.',
 'Z tej właśnie przyczyny przypominam ci, abyś rozpalił na nowo charyzmat Boży, który jest w tobie przez włożenie moich rąk. Albowiem nie dał nam Bóg ducha bojaźni, ale mocy i miłości, i trzeźwego myślenia. Nie wstydź się zatem świadectwa naszego Pana ani mnie, Jego więźnia, lecz weź udz

In [ ]:
content_dic['EWANGELIA']

['Łk 10,1-9',
 'Rozesłanie uczniów',
 'Słowa Ewangelii według świętego Łukasza',
 'Spośród swoich uczniów wyznaczył Pan jeszcze innych, siedemdziesięciu dwóch, i wysłał ich po dwóch przed sobą do każdego miasta i miejscowości, dokąd sam przyjść zamierzał.',
 'Powiedział też do nich: „Żniwo wprawdzie wielkie, ale robotników mało; proście więc Pana żniwa, żeby wyprawił robotników na swoje żniwo. Idźcie, oto was posyłam jak owce między wilki. Nie noście ze sobą trzosa ani torby, ani sandałów; i nikogo w drodze nie pozdrawiajcie.',
 'Gdy do jakiego domu wejdziecie, najpierw mówcie: «Pokój temu domowi!» Jeśli tam mieszka człowiek godny pokoju, wasz pokój spocznie na nim; jeśli nie, powróci do was.',
 'W tym samym domu zostańcie, jedząc i pijąc, co mają: bo zasługuje robotnik na swą zapłatę.',
 'Nie przechodźcie z domu do domu. Jeśli do jakiego miasta wejdziecie i przyjmą was, jedzcie, co wam podadzą; uzdrawiajcie chorych, którzy tam są, i mówcie im: «Przybliżyło się do was królestwo Boże»”.

In [ ]:
URL = f"https://www.vaticannews.va/en/word-of-the-day/{str(today).replace('-', '/')}.html"

page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
content_list = soup.find_all("div", "section__content")[:2]
content_list_text = [x.get_text().split('\n') for x in content_list]
for num, y in enumerate(content_list_text):
    content_list_text[num] = [x.replace("\"","") for x in y]

content_list_text

[['',
  'A reading from the Second Letter of St. Paul to Timothy 2 Tm 1:1-8',
  'Paul, an Apostle of Christ Jesus by the will of God for the promise of life in Christ Jesus, to Timothy, my dear child: grace, mercy, and peace from God the Father and Christ Jesus our Lord.',
  'I am grateful to God, whom I worship with a clear conscience as my ancestors did, as I remember you constantly in my prayers, night and day. I yearn to see you again, recalling your tears, so that I may be filled with joy, as I recall your sincere faith that first lived in your grandmother Lois and in your mother Eunice and that I am confident lives also in you.',
  'For this reason, I remind you to stir into flame the gift of God that you have through the imposition of my hands. For God did not give us a spirit of cowardice but rather of power and love and self-control. So do not be ashamed of your testimony to our Lord, nor of me, a prisoner for his sake; but bear your share of hardship for the Gospel with the s

In [ ]:
str(today)

'2024-01-26'